MovieLensを利用してサンプルを作成する  
MovieLensのダウンロード

- genome-scores: 
- genome-tags: 
- links: 
- movies: 映画のマスタデータ
- ratings: ユーザーによる評価データ
- tags: 

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.dirname(Path().resolve()))
from datasets.movie_lens import MovieLensDataset

In [2]:
movielens_datasets = MovieLensDataset()

In [3]:
movielens_datasets.ratings.head()

,userId,movieId,rating,timestamp,userIdx,movieIdx
0,1,307,3.5,1256677221,0,304
1,6,307,4.0,832059248,5,304
2,56,307,4.0,1383625728,55,304
3,71,307,5.0,1257795414,70,304
4,84,307,3.0,999055519,83,304


# TensorFlowを使う

In [4]:
!pip install tensorflow


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import tensorflow as tf
from collections import Counter
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: cannot open shared object file: No such file or directory']
  warnings.warn(f"file system 

In [6]:
def create_model(user_num: int, item_num: int, dimension: int = 10) -> tf.keras.models.Model:
    user_input_layer = tf.keras.layers.Input(shape=(1, ))
    item_input_layer = tf.keras.layers.Input(shape=(1, ))
    user_embedding_layer = tf.keras.layers.Embedding(user_num, dimension)(user_input_layer)
    item_embedding_layer = tf.keras.layers.Embedding(item_num, dimension)(item_input_layer)
    user_bias_layer = tf.keras.layers.Embedding(user_num, 1)(user_input_layer)
    item_bias_layer = tf.keras.layers.Embedding(item_num, 1)(item_input_layer)

    x = tf.keras.layers.Dot(axes=2)([user_embedding_layer, item_embedding_layer])
    x = tf.keras.layers.Add()([x, user_bias_layer, item_bias_layer])
    x = tf.keras.layers.Flatten()(x)

    model = tf.keras.models.Model(inputs=[user_input_layer, item_input_layer], outputs=x)
    opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        optimizer=opt,
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )

    return model

In [7]:
# 学習データと検証データの用意
train, test = train_test_split(movielens_datasets.ratings, test_size=0.3, shuffle=True, random_state=42)
print(train.shape, test.shape)

(19427410, 6) (8326034, 6)


In [8]:
# モデルの作成
user_num = len(movielens_datasets.users)
content_num = len(movielens_datasets.movies)
model = create_model(user_num, content_num)

In [9]:
# 学習
tf.keras.utils.plot_model(model, show_shapes=True)

# callback関数
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=0
    )
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'keras.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    verbose=0
)

# 学習実行
result = model.fit(
    x=[train['userIdx'].values, train['movieIdx'].values],
    y=train['rating'].values,
    epochs=200,
    batch_size=1024,
    validation_data=(
        [test['userIdx'].values, test['movieIdx'].values],
        test['rating'].values
    ),
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Epoch 1/200
18973/18973 [==============================] - 82s 4ms/step - loss: 2.0103 - root_mean_squared_error: 1.4178 - val_loss: 1.2443 - val_root_mean_squared_error: 1.1155
Epoch 2/200
18973/18973 [==============================] - 83s 4ms/step - loss: 1.1343 - root_mean_squared_error: 1.0650 - val_loss: 1.0667 - val_root_mean_squared_error: 1.0328
Epoch 3/200
18973/18973 [==============================] - 84s 4ms/step - loss: 1.0175 - root_mean_squared_error: 1.0087 - val_loss: 0.9910 - val_root_mean_squared_error: 0.9955
Epoch 4/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.9584 - root_mean_squared_error: 0.9790 - val_loss: 0.9470 - val_root_mean_squared_error: 0.9731
Epoch 5/200
18973/18973 [==============================] - 86s 5ms/step - loss: 0.9205 - root_mean_squared_error: 0.9594 - val_loss: 0.9159

Epoch 46/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.5396 - root_mean_squared_error: 0.7346 - val_loss: 0.6639 - val_root_mean_squared_error: 0.8148
Epoch 47/200
18973/18973 [==============================] - 83s 4ms/step - loss: 0.5383 - root_mean_squared_error: 0.7337 - val_loss: 0.6636 - val_root_mean_squared_error: 0.8146
Epoch 48/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.5372 - root_mean_squared_error: 0.7329 - val_loss: 0.6635 - val_root_mean_squared_error: 0.8145
Epoch 49/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.5360 - root_mean_squared_error: 0.7321 - val_loss: 0.6634 - val_root_mean_squared_error: 0.8145
Epoch 50/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.5349 - root_mean_squared_error: 0.7314 - val_loss: 0.6633 - val_root_mean_squared_error: 0.8144
Epoch 51/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.5339 - root_mean_squar

In [10]:
result

In [11]:
model.save('../datasets/matrix-factorization.h5')

In [28]:
len(test.userIdx.values)

8326034

In [29]:
test.userIdx.values
model.predict([np.array([283214, 283214, 283214]), np.array([1, 2, 3])], verbose=1)

1/1 [==============================] - 0s 36ms/step


array([[3.952248 ],
       [3.679933 ],
       [3.3663383]], dtype=float32)

In [20]:
movielens_datasets.ratings.query('movieIdx == 2')

,userId,movieId,rating,timestamp,userIdx,movieIdx
12088389,8,3,3.0,858269342,7,2
12088390,19,3,4.0,836845013,18,2
12088391,51,3,4.0,922259635,50,2
12088392,81,3,3.5,1280373800,80,2
12088393,92,3,1.0,1132007480,91,2
...,...,...,...,...,...,...
12103969,283192,3,3.0,859281419,283191,2
12103970,283195,3,4.5,1078372373,283194,2
12103971,283196,3,2.5,1132961005,283195,2
12103972,283215,3,5.0,854982147,283214,2


In [10]:
user_ids = ratings['userId'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = ratings['movieId'].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
ratings['user'] = ratings['userId'].map(user2user_encoded)
ratings['movie'] = ratings['movieId'].map(movie2movie_encoded)

In [15]:
num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
ratings['rating'] = ratings['rating'].values.astype(np.float32)

min_rating = min(ratings['rating'])
max_rating = max(ratings['rating'])

In [16]:
ratings

,userId,movieId,rating,timestamp,user,movie
0,1,307,3.5,1256677221,0,0
1,1,481,3.5,1256677456,0,1
2,1,1091,1.5,1256677471,0,2
3,1,1257,4.5,1256677460,0,3
4,1,1449,4.5,1256677264,0,4
...,...,...,...,...,...,...
27753439,283228,8542,4.5,1379882795,283227,9793
27753440,283228,8712,4.5,1379882751,283227,7826
27753441,283228,34405,4.5,1379882889,283227,1080
27753442,283228,44761,4.5,1354159524,283227,4306


In [17]:
_ratings = ratings.sample(frac=1, random_state=42)
x = _ratings[['user', 'movie']].values
# Normalize the targets between 0 and 1. Makes it easy to train.
y = _ratings['rating'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values

In [18]:
train_indices = int(0.9 * _ratings.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

In [19]:
print(x_train[0].shape)
print(x_train[0])
print(y_train[0])

(2,)
[268199    316]
1.0


# Matrix Factorizationを試す

## 評価値行列の作成
MoviewLensのRatingデータからユーザーによるアイテム（映画）の評価値行列を作成する

- ユーザーとアイテムのindex化
- ユーザー$u$のアイテム$i$に対するの評価値が、$u$行$i$列の要素となる行列を作成する。

In [78]:
# ユーザーとアイテムのindex化
user_ids = ratings.groupby('userId').size().sort_values(ascending=False).index
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = ratings.groupby('movieId').size().sort_values(ascending=False).index
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}

In [79]:
# 評価値行列の初期化
R = np.zeros((len(user_ids), len(movie_ids)))
R.shape

(283228, 53889)

In [80]:
# 評価値行列の生成
for user_id, movie_id, rating in ratings[['userId', 'movieId', 'rating']].values:
    R[user2user_encoded[user_id]][movie2movie_encoded[movie_id]] = rating
    
R

array([[4.5, 3.5, 4.5, ..., 0. , 0. , 0. ],
       [4. , 4. , 4.5, ..., 0. , 0. , 0. ],
       [4.5, 3. , 4. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [89]:
# 因子分解の関数を定義
# パラメータ
#     R : 正解データとなるUser, Itemの評価値行列
#     K : 次元数
#     steps : 学習回数
#     alpha : 学習率
#     beta : 正則化パラメータ
def matrix_factorization(R, K, steps=5000, alpha=0.0002, beta=0.02):
    row, column = R.shape
    
    # ユーザーの因子行列
    P = np.random.rand(row, K)
    # アイテムの因子行列
    Q = np.random.rand(column, K).T

    for step in range(steps):
        print(f'学習{step + 1}回目')
        for r in range(row):
            for c in range(column):
                if R[r][c] <= 0:
                    next
                # 勾配計算
                e_rc = R[r][c] - np.dot(P[r, :], Q[:, c])
                for k in range(K):
#                     P[r][k] = P[r][k] + alpha * -2 * e_rc * Q[k][c]
#                     Q[k][c] = Q[k][c] + alpha * -2 * e_rc * P[r][k]
                    
                    P[r][k] = P[r][k] + alpha * (2 * e_rc * Q[k][c] - beta * P[r][k])
                    Q[k][c] = Q[k][c] + alpha * (2 * e_rc * P[r][k] - beta * Q[k][c])
        
        # ステップ完了後の推論行列R
        eR = np.dot(P, Q)

        # 学習後の誤差の計算
        e = 0
        for r in range(row):
            for c in range(column):
                if R[r][c] <= 0:
                    next

                e = e + pow(R[r][c] - np.dot(P[r, :], Q[:, c]), 2)
                for k in range(K):
                    e = e + (beta / 2) * (pow(P[r][k], 2) + pow(Q[k][c], 2))
        print(f'  誤差計算完了 （誤差：{e}）')

        if e < 0.001:
            break
    return P, Q.T

In [90]:
R[0:100, 0:100]

array([[4.5, 3.5, 4.5, ..., 3.5, 4. , 3. ],
       [4. , 4. , 4.5, ..., 4. , 4. , 4. ],
       [4.5, 3. , 4. , ..., 3.5, 5. , 3.5],
       ...,
       [3.5, 4. , 4. , ..., 0. , 3.5, 4. ],
       [3. , 3. , 5. , ..., 3. , 2. , 3. ],
       [5. , 4. , 4.5, ..., 4. , 4.5, 3.5]])

In [91]:
P, Q = matrix_factorization(R[0:100, 0:100], 2)

学習1回目
  誤差計算完了 （誤差：101772.222112844）
学習2回目
  誤差計算完了 （誤差：93499.89298090492）
学習3回目
  誤差計算完了 （誤差：84301.21060576847）
学習4回目
  誤差計算完了 （誤差：74462.2691908962）
学習5回目
  誤差計算完了 （誤差：64395.36272174002）
学習6回目
  誤差計算完了 （誤差：54588.63115298788）
学習7回目
  誤差計算完了 （誤差：45523.370839086994）
学習8回目
  誤差計算完了 （誤差：37583.65946615742）
学習9回目
  誤差計算完了 （誤差：30991.15011173036）
学習10回目
  誤差計算完了 （誤差：25787.731780223236）
学習11回目
  誤差計算完了 （誤差：21865.82588330384）
学習12回目
  誤差計算完了 （誤差：19026.136853379496）
学習13回目
  誤差計算完了 （誤差：17037.32682964604）
学習14回目
  誤差計算完了 （誤差：15680.202069156501）
学習15回目
  誤差計算完了 （誤差：14771.340861360726）
学習16回目
  誤差計算完了 （誤差：14169.842115560896）
学習17回目
  誤差計算完了 （誤差：13773.920464439998）
学習18回目
  誤差計算完了 （誤差：13513.241428378236）
学習19回目
  誤差計算完了 （誤差：13340.709047398796）
学習20回目
  誤差計算完了 （誤差：13225.45025200445）
学習21回目
  誤差計算完了 （誤差：13147.4860149361）
学習22回目
  誤差計算完了 （誤差：13093.964610521585）
学習23回目
  誤差計算完了 （誤差：13056.6211401378）
学習24回目
  誤差計算完了 （誤差：13030.116492784826）
学習25回目
  誤差計算完了 （誤差：13010.972757771628）
学習26回目
  誤差計算完了 （誤差：12996.

  誤差計算完了 （誤差：12542.467408787521）
学習206回目
  誤差計算完了 （誤差：12535.581618517615）
学習207回目
  誤差計算完了 （誤差：12528.592583325997）
学習208回目
  誤差計算完了 （誤差：12521.499893249422）
学習209回目
  誤差計算完了 （誤差：12514.303195071063）
学習210回目
  誤差計算完了 （誤差：12507.002194667451）
学習211回目
  誤差計算完了 （誤差：12499.596659354032）
学習212回目
  誤差計算完了 （誤差：12492.086420219897）
学習213回目
  誤差計算完了 （誤差：12484.471374447901）
学習214回目
  誤差計算完了 （誤差：12476.751487612059）
学習215回目
  誤差計算完了 （誤差：12468.926795947242）
学習216回目
  誤差計算完了 （誤差：12460.997408582678）
学習217回目
  誤差計算完了 （誤差：12452.96350973274）
学習218回目
  誤差計算完了 （誤差：12444.825360838138）
学習219回目
  誤差計算完了 （誤差：12436.583302647381）
学習220回目
  誤差計算完了 （誤差：12428.237757234765）
学習221回目
  誤差計算完了 （誤差：12419.789229942171）
学習222回目
  誤差計算完了 （誤差：12411.238311240128）
学習223回目
  誤差計算完了 （誤差：12402.585678497255）
学習224回目
  誤差計算完了 （誤差：12393.832097651713）
学習225回目
  誤差計算完了 （誤差：12384.978424773506）
学習226回目
  誤差計算完了 （誤差：12376.025607512101）
学習227回目
  誤差計算完了 （誤差：12366.974686417623）
学習228回目
  誤差計算完了 （誤差：12357.826796130372）
学習229回目
  誤差計算完了 （誤差：1234

  誤差計算完了 （誤差：11137.692937356116）
学習409回目
  誤差計算完了 （誤差：11136.077615218393）
学習410回目
  誤差計算完了 （誤差：11134.486348144703）
学習411回目
  誤差計算完了 （誤差：11132.91871930273）
学習412回目
  誤差計算完了 （誤差：11131.374320317185）
学習413回目
  誤差計算完了 （誤差：11129.852751083954）
学習414回目
  誤差計算完了 （誤差：11128.353619587237）
学習415回目
  誤差計算完了 （誤差：11126.876541719375）
学習416回目
  誤差計算完了 （誤差：11125.421141105619）
学習417回目
  誤差計算完了 （誤差：11123.987048930412）
学習418回目
  誤差計算完了 （誤差：11122.573903768971）
学習419回目
  誤差計算完了 （誤差：11121.181351420924）
学習420回目
  誤差計算完了 （誤差：11119.809044748295）
学習421回目
  誤差計算完了 （誤差：11118.456643516334）
学習422回目
  誤差計算完了 （誤差：11117.12381423846）
学習423回目
  誤差計算完了 （誤差：11115.810230023753）
学習424回目
  誤差計算完了 （誤差：11114.515570428272）
学習425回目
  誤差計算完了 （誤差：11113.239521309573）
学習426回目
  誤差計算完了 （誤差：11111.98177468435）
学習427回目
  誤差計算完了 （誤差：11110.742028590324）
学習428回目
  誤差計算完了 （誤差：11109.519986949375）
学習429回目
  誤差計算完了 （誤差：11108.31535943592）
学習430回目
  誤差計算完了 （誤差：11107.12786134623）
学習431回目
  誤差計算完了 （誤差：11105.957213473626）
学習432回目
  誤差計算完了 （誤差：11104.80

  誤差計算完了 （誤差：11026.389859972402）
学習613回目
  誤差計算完了 （誤差：11026.274115657636）
学習614回目
  誤差計算完了 （誤差：11026.159734307534）
学習615回目
  誤差計算完了 （誤差：11026.046699301412）
学習616回目
  誤差計算完了 （誤差：11025.934994232533）
学習617回目
  誤差計算完了 （誤差：11025.824602904779）
学習618回目
  誤差計算完了 （誤差：11025.715509329562）
学習619回目
  誤差計算完了 （誤差：11025.60769772352）
学習620回目
  誤差計算完了 （誤差：11025.501152505425）
学習621回目
  誤差計算完了 （誤差：11025.39585829317）
学習622回目
  誤差計算完了 （誤差：11025.291799901077）
学習623回目
  誤差計算完了 （誤差：11025.188962337752）
学習624回目
  誤差計算完了 （誤差：11025.087330802899）
学習625回目
  誤差計算完了 （誤差：11024.986890684511）
学習626回目
  誤差計算完了 （誤差：11024.88762755687）
学習627回目
  誤差計算完了 （誤差：11024.789527177574）
学習628回目
  誤差計算完了 （誤差：11024.692575485149）
学習629回目
  誤差計算完了 （誤差：11024.596758596717）
学習630回目
  誤差計算完了 （誤差：11024.502062805412）
学習631回目
  誤差計算完了 （誤差：11024.408474577931）
学習632回目
  誤差計算完了 （誤差：11024.315980552368）
学習633回目
  誤差計算完了 （誤差：11024.224567535797）
学習634回目
  誤差計算完了 （誤差：11024.134222501809）
学習635回目
  誤差計算完了 （誤差：11024.044932588658）
学習636回目
  誤差計算完了 （誤差：11023.

  誤差計算完了 （誤差：11017.32436602111）
学習817回目
  誤差計算完了 （誤差：11017.313335776109）
学習818回目
  誤差計算完了 （誤差：11017.302429892456）
学習819回目
  誤差計算完了 （誤差：11017.291646961212）
学習820回目
  誤差計算完了 （誤差：11017.280985589201）
学習821回目
  誤差計算完了 （誤差：11017.270444399232）
学習822回目
  誤差計算完了 （誤差：11017.260022030112）
学習823回目
  誤差計算完了 （誤差：11017.24971713647）
学習824回目
  誤差計算完了 （誤差：11017.239528387934）
学習825回目
  誤差計算完了 （誤差：11017.229454469889）
学習826回目
  誤差計算完了 （誤差：11017.219494082507）
学習827回目
  誤差計算完了 （誤差：11017.209645941199）
学習828回目
  誤差計算完了 （誤差：11017.199908775707）
学習829回目
  誤差計算完了 （誤差：11017.190281330963）
学習830回目
  誤差計算完了 （誤差：11017.180762365682）
学習831回目
  誤差計算完了 （誤差：11017.171350653165）
学習832回目
  誤差計算完了 （誤差：11017.162044980767）
学習833回目
  誤差計算完了 （誤差：11017.152844149767）
学習834回目
  誤差計算完了 （誤差：11017.143746974942）
学習835回目
  誤差計算完了 （誤差：11017.134752285023）
学習836回目
  誤差計算完了 （誤差：11017.125858922056）
学習837回目
  誤差計算完了 （誤差：11017.117065741282）
学習838回目
  誤差計算完了 （誤差：11017.108371610751）
学習839回目
  誤差計算完了 （誤差：11017.09977541234）
学習840回目
  誤差計算完了 （誤差：11017.

...

  誤差計算完了 （誤差：11016.422922915663）
学習4185回目
  誤差計算完了 （誤差：11016.422930107074）
学習4186回目
  誤差計算完了 （誤差：11016.422937293075）
学習4187回目
  誤差計算完了 （誤差：11016.422944473496）
学習4188回目
  誤差計算完了 （誤差：11016.422951648578）
学習4189回目
  誤差計算完了 （誤差：11016.422958818015）
学習4190回目
  誤差計算完了 （誤差：11016.422965982067）
学習4191回目
  誤差計算完了 （誤差：11016.422973140743）
学習4192回目
  誤差計算完了 （誤差：11016.42298029387）
学習4193回目
  誤差計算完了 （誤差：11016.422987441589）
学習4194回目
  誤差計算完了 （誤差：11016.422994583856）
学習4195回目
  誤差計算完了 （誤差：11016.423001720597）
学習4196回目
  誤差計算完了 （誤差：11016.423008851867）
学習4197回目
  誤差計算完了 （誤差：11016.423015977862）
学習4198回目
  誤差計算完了 （誤差：11016.423023098301）
学習4199回目
  誤差計算完了 （誤差：11016.42303021342）
学習4200回目
  誤差計算完了 （誤差：11016.423037322887）
学習4201回目
  誤差計算完了 （誤差：11016.423044427123）
学習4202回目
  誤差計算完了 （誤差：11016.423051525844）
学習4203回目
  誤差計算完了 （誤差：11016.423058619339）
学習4204回目
  誤差計算完了 （誤差：11016.423065707255）
学習4205回目
  誤差計算完了 （誤差：11016.423072789725）
学習4206回目
  誤差計算完了 （誤差：11016.423079866889）
学習4207回目
  誤差計算完了 （誤差：11016.423086938554）
学習4

  誤差計算完了 （誤差：11016.424232415891）
学習4381回目
  誤差計算完了 （誤差：11016.42423860537）
学習4382回目
  誤差計算完了 （誤差：11016.4242447902）
学習4383回目
  誤差計算完了 （誤差：11016.424250970147）
学習4384回目
  誤差計算完了 （誤差：11016.424257145429）
学習4385回目
  誤差計算完了 （誤差：11016.424263316092）
学習4386回目
  誤差計算完了 （誤差：11016.424269481826）
学習4387回目
  誤差計算完了 （誤差：11016.424275642948）
学習4388回目
  誤差計算完了 （誤差：11016.424281799356）
学習4389回目
  誤差計算完了 （誤差：11016.424287950975）
学習4390回目
  誤差計算完了 （誤差：11016.424294097844）
学習4391回目
  誤差計算完了 （誤差：11016.424300240069）
学習4392回目
  誤差計算完了 （誤差：11016.424306377576）
学習4393回目
  誤差計算完了 （誤差：11016.424312510395）
学習4394回目
  誤差計算完了 （誤差：11016.4243186383）
学習4395回目
  誤差計算完了 （誤差：11016.424324761701）
学習4396回目
  誤差計算完了 （誤差：11016.424330880245）
学習4397回目
  誤差計算完了 （誤差：11016.424336994234）
学習4398回目
  誤差計算完了 （誤差：11016.4243431034）
学習4399回目
  誤差計算完了 （誤差：11016.424349207862）
学習4400回目
  誤差計算完了 （誤差：11016.424355307714）
学習4401回目
  誤差計算完了 （誤差：11016.424361403011）
学習4402回目
  誤差計算完了 （誤差：11016.424367493522）
学習4403回目
  誤差計算完了 （誤差：11016.424373579182）
学習4404回目

  誤差計算完了 （誤差：11016.425358927489）
学習4577回目
  誤差計算完了 （誤差：11016.425364249533）
学習4578回目
  誤差計算完了 （誤差：11016.425369567485）
学習4579回目
  誤差計算完了 （誤差：11016.425374881299）
学習4580回目
  誤差計算完了 （誤差：11016.425380190954）
学習4581回目
  誤差計算完了 （誤差：11016.4253854965）
学習4582回目
  誤差計算完了 （誤差：11016.425390797978）
学習4583回目
  誤差計算完了 （誤差：11016.425396095414）
学習4584回目
  誤差計算完了 （誤差：11016.425401388593）
学習4585回目
  誤差計算完了 （誤差：11016.425406677889）
学習4586回目
  誤差計算完了 （誤差：11016.425411963057）
学習4587回目
  誤差計算完了 （誤差：11016.42541724401）
学習4588回目
  誤差計算完了 （誤差：11016.425422520995）
学習4589回目
  誤差計算完了 （誤差：11016.42542779383）
学習4590回目
  誤差計算完了 （誤差：11016.425433062726）
学習4591回目
  誤差計算完了 （誤差：11016.425438327416）
学習4592回目
  誤差計算完了 （誤差：11016.425443588032）
学習4593回目
  誤差計算完了 （誤差：11016.425448844728）
学習4594回目
  誤差計算完了 （誤差：11016.425454097114）
学習4595回目
  誤差計算完了 （誤差：11016.425459345686）
学習4596回目
  誤差計算完了 （誤差：11016.425464590086）
学習4597回目
  誤差計算完了 （誤差：11016.425469830448）
学習4598回目
  誤差計算完了 （誤差：11016.42547506676）
学習4599回目
  誤差計算完了 （誤差：11016.425480299007）
学習4600

  誤差計算完了 （誤差：11016.42634541043）
学習4777回目
  誤差計算完了 （誤差：11016.426349968435）
学習4778回目
  誤差計算完了 （誤差：11016.42635452302）
学習4779回目
  誤差計算完了 （誤差：11016.426359074105）
学習4780回目
  誤差計算完了 （誤差：11016.426363621651）
学習4781回目
  誤差計算完了 （誤差：11016.426368165326）
学習4782回目
  誤差計算完了 （誤差：11016.426372705813）
学習4783回目
  誤差計算完了 （誤差：11016.426377242842）
学習4784回目
  誤差計算完了 （誤差：11016.42638177603）
学習4785回目
  誤差計算完了 （誤差：11016.426386305826）
学習4786回目
  誤差計算完了 （誤差：11016.426390832183）
学習4787回目
  誤差計算完了 （誤差：11016.426395354998）
学習4788回目
  誤差計算完了 （誤差：11016.426399874375）
学習4789回目
  誤差計算完了 （誤差：11016.426404390137）
学習4790回目
  誤差計算完了 （誤差：11016.426408902378）
学習4791回目
  誤差計算完了 （誤差：11016.426413411142）
学習4792回目
  誤差計算完了 （誤差：11016.426417916322）
学習4793回目
  誤差計算完了 （誤差：11016.42642241829）
学習4794回目
  誤差計算完了 （誤差：11016.426426916416）
学習4795回目
  誤差計算完了 （誤差：11016.426431411199）
学習4796回目
  誤差計算完了 （誤差：11016.426435902522）
学習4797回目
  誤差計算完了 （誤差：11016.426440390229）
学習4798回目
  誤差計算完了 （誤差：11016.426444874629）
学習4799回目
  誤差計算完了 （誤差：11016.42644935546）
学習4800

  誤差計算完了 （誤差：11016.427189983557）
学習4977回目
  誤差計算完了 （誤差：11016.427193884638）
学習4978回目
  誤差計算完了 （誤差：11016.42719778249）
学習4979回目
  誤差計算完了 （誤差：11016.427201677285）
学習4980回目
  誤差計算完了 （誤差：11016.427205569109）
学習4981回目
  誤差計算完了 （誤差：11016.427209457868）
学習4982回目
  誤差計算完了 （誤差：11016.427213343713）
学習4983回目
  誤差計算完了 （誤差：11016.427217226388）
学習4984回目
  誤差計算完了 （誤差：11016.427221106102）
学習4985回目
  誤差計算完了 （誤差：11016.427224982841）
学習4986回目
  誤差計算完了 （誤差：11016.427228856552）
学習4987回目
  誤差計算完了 （誤差：11016.4272327271）
学習4988回目
  誤差計算完了 （誤差：11016.42723659475）
学習4989回目
  誤差計算完了 （誤差：11016.427240459268）
学習4990回目
  誤差計算完了 （誤差：11016.427244320872）
学習4991回目
  誤差計算完了 （誤差：11016.427248179467）
学習4992回目
  誤差計算完了 （誤差：11016.427252034993）
学習4993回目
  誤差計算完了 （誤差：11016.427255887547）
学習4994回目
  誤差計算完了 （誤差：11016.42725973722）
学習4995回目
  誤差計算完了 （誤差：11016.427263583639）
学習4996回目
  誤差計算完了 （誤差：11016.427267427196）
学習4997回目
  誤差計算完了 （誤差：11016.427271267823）
学習4998回目
  誤差計算完了 （誤差：11016.427275105301）
学習4999回目
  誤差計算完了 （誤差：11016.42727893983）
学習5000回

In [67]:
np.dot(P, Q.T)

array([[ 7.06891157e-02,  4.21200609e-02,  1.02516102e-02, ...,
         1.70907570e-01,  2.41873983e-01,  4.20820799e-03],
       [ 1.06792309e-01, -6.78030379e-02, -2.08539610e-03, ...,
         1.81480780e-01,  1.45273048e-01, -2.70170721e-03],
       [ 1.07007124e-01, -1.64667997e-02,  4.79227974e-03, ...,
         2.11888858e-01,  2.31774014e-01,  8.40614115e-04],
       ...,
       [ 3.33017526e-03,  3.34728246e-02,  4.69296115e-03, ...,
         2.64303976e-02,  6.41331946e-02,  2.36860024e-03],
       [ 1.20614753e-01, -7.51147983e-02, -2.15955380e-03, ...,
         2.05824976e-01,  1.66528456e-01, -2.95047791e-03],
       [ 7.10611578e-01,  5.91639187e-01,  1.25546839e-01, ...,
         1.81625668e+00,  2.71321548e+00,  5.38982632e-02]])

# TensorFlowを使う

In [92]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 MB 8.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 13.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 14.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 14.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 13.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [97]:
import tensorflow as tf
from collections import Counter
from sklearn.model_selection import train_test_split

In [95]:
def create_model(user_num: int, movie_num: int, k: int = 10) -> tf.keras.models.Model:
    """kerasでMatrix Factorizationのモデルを構築する

    Args:
        user_num (int): ユニークユーザー数
        movie_num (int): ユニーク映画数
        k (int): 埋め込み層の次元数

    Returns:
        tf.keras.models.Model: モデルインスタンス
    """

    u = tf.keras.layers.Input(shape=(1,))
    m = tf.keras.layers.Input(shape=(1,))
    u_embedding = tf.keras.layers.Embedding(user_num, k)(u)
    m_embedding = tf.keras.layers.Embedding(movie_num, k)(m)
    u_bias = tf.keras.layers.Embedding(user_num, 1)(u)
    m_bias = tf.keras.layers.Embedding(movie_num, 1)(m)

    x = tf.keras.layers.Dot(axes=2)([u_embedding, m_embedding])
    x = tf.keras.layers.Add()([x, u_bias, m_bias])
    x = tf.keras.layers.Flatten()(x)

    model = tf.keras.models.Model(inputs=[u, m], outputs=x)
    opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        optimizer=opt,
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )

    return model

In [100]:
# 出現回数の多いuserとmovieに絞る
n = 10000  # userはTOP:10000
m = 2000  # movieはTOP:2000
user_ids_count = Counter(ratings.userId)
movie_ids_count = Counter(ratings.movieId)
user_ids = [u for u, c in user_ids_count.most_common(n)]
movie_ids = [m for m, c in movie_ids_count.most_common(m)]
ratings_small = ratings[ratings.userId.isin(user_ids) & ratings.movieId.isin(movie_ids)]

# indexを付与する
user_id_map = {}
for i, u_id in enumerate(user_ids):
    user_id_map[u_id] = i

movie_id_map = {}
for i, m_id in enumerate(movie_ids):
    movie_id_map[m_id] = i

ratings_small.loc[:, 'user_idx'] = ratings_small.apply(lambda row: user_id_map[row.userId], axis=1)
ratings_small.loc[:, 'movie_idx'] = ratings_small.apply(lambda row: movie_id_map[row.movieId], axis=1)

/tmp/ipykernel_11/762044213.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_small.loc[:, 'user_idx'] = ratings_small.apply(lambda row: user_id_map[row.userId], axis=1)
/tmp/ipykernel_11/762044213.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_small.loc[:, 'movie_idx'] = ratings_small.apply(lambda row: movie_id_map[row.movieId], axis=1)


In [101]:
ratings_small

,userId,movieId,rating,timestamp,user,movie,user_idx,movie_idx
42,4,1,4.0,1113765937,3,42,5126,9
43,4,2,4.0,1113767306,3,43,5126,135
44,4,5,2.0,1123990453,3,44,5126,366
45,4,6,4.5,1113767242,3,45,5126,124
46,4,10,4.0,1113765995,3,46,5126,91
...,...,...,...,...,...,...,...,...
27751532,283195,91500,4.0,1334813573,283194,1112,901,448
27751534,283195,95167,4.0,1343232598,283194,1010,901,1134
27751538,283195,103141,4.0,1378959042,283194,1032,901,1554
27751539,283195,103253,1.5,1395469745,283194,2951,901,1358


In [104]:
user_num = ratings_small.user_idx.max() + 1 # number of users
movie_num = ratings_small.movie_idx.max() + 1 # number of movies

print(user_num, movie_num)
# -> 10000 2000

# train, testの分割
train, test = train_test_split(ratings_small, test_size=0.3, shuffle=True, random_state=42)
print(train.shape, test.shape)

10000 2000
(4102638, 8) (1758274, 8)


In [105]:
# モデルの定義
model = create_model(user_num, movie_num)
tf.keras.utils.plot_model(model, show_shapes=True)  # ネットワーク構造をプロットできる

# callback関数を定義
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=0,
    )
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'keras.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    verbose=0,
)

# 学習
result = model.fit(
    x=[train.user_idx.values, train.movie_idx.values],
    y=train.rating.values,
    epochs=200,
    batch_size=1024,
    validation_data=(
        [test.user_idx.values, test.movie_idx.values],
        test.rating.values
    ),
    callbacks=[early_stopping, checkpoint],
    verbose=1,
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Epoch 1/200
4007/4007 [==============================] - 7s 2ms/step - loss: 1.9683 - root_mean_squared_error: 1.4030 - val_loss: 0.7485 - val_root_mean_squared_error: 0.8651
Epoch 2/200
4007/4007 [==============================] - 6s 2ms/step - loss: 0.7062 - root_mean_squared_error: 0.8403 - val_loss: 0.6878 - val_root_mean_squared_error: 0.8293
Epoch 3/200
4007/4007 [==============================] - 6s 2ms/step - loss: 0.6796 - root_mean_squared_error: 0.8244 - val_loss: 0.6781 - val_root_mean_squared_error: 0.8234
Epoch 4/200
4007/4007 [==============================] - 6s 2ms/step - loss: 0.6743 - root_mean_squared_error: 0.8211 - val_loss: 0.6756 - val_root_mean_squared_error: 0.8219
Epoch 5/200
4007/4007 [==============================] - 6s 2ms/step - loss: 0.6727 - root_mean_squared_error: 0.8202 - val_loss: 0.6749 - val_root_mea

Epoch 47/200
4007/4007 [==============================] - 7s 2ms/step - loss: 0.5068 - root_mean_squared_error: 0.7119 - val_loss: 0.5400 - val_root_mean_squared_error: 0.7348
Epoch 48/200
4007/4007 [==============================] - 7s 2ms/step - loss: 0.5065 - root_mean_squared_error: 0.7117 - val_loss: 0.5399 - val_root_mean_squared_error: 0.7348
Epoch 49/200
4007/4007 [==============================] - 7s 2ms/step - loss: 0.5062 - root_mean_squared_error: 0.7115 - val_loss: 0.5396 - val_root_mean_squared_error: 0.7346
Epoch 50/200
4007/4007 [==============================] - 7s 2ms/step - loss: 0.5059 - root_mean_squared_error: 0.7113 - val_loss: 0.5395 - val_root_mean_squared_error: 0.7345
Epoch 51/200
4007/4007 [==============================] - 7s 2ms/step - loss: 0.5057 - root_mean_squared_error: 0.7111 - val_loss: 0.5394 - val_root_mean_squared_error: 0.7344
Epoch 52/200
4007/4007 [==============================] - 7s 2ms/step - loss: 0.5055 - root_mean_squared_error: 0.7110 -

In [106]:
train = pd.merge(train, movies, how='left', on='movieId')
test = pd.merge(test, movies, how='left', on='movieId')

In [107]:
test.loc[:, 'pred_rating'] = model.predict([test.user_idx.values, test.movie_idx.values], verbose=1)

54947/54947 [==============================] - 37s 681us/step


In [108]:
test

,userId,movieId,rating,timestamp,user,movie,user_idx,movie_idx,title,genres,pred_rating
0,15281,2011,3.5,1441344444,15280,316,7766,160,Back to the Future Part II (1989),Adventure|Comedy|Sci-Fi,3.771882
1,54738,4886,4.5,1463958935,54737,566,7818,70,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,3.932548
2,110305,2381,1.0,1424801960,110304,356,7749,1883,Police Academy 4: Citizens on Patrol (1987),Comedy|Crime,1.906974
3,117113,2294,4.0,1005172889,117112,346,1574,486,Antz (1998),Adventure|Animation|Children|Comedy|Fantasy,3.678500
4,232000,8961,3.0,1115071058,231999,711,2344,101,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,4.330770
...,...,...,...,...,...,...,...,...,...,...,...
1758269,7753,762,1.0,998065932,7752,177,9298,565,Striptease (1996),Comedy|Crime,0.181198
1758270,181585,1387,2.5,1502734330,181584,1394,9944,146,Jaws (1975),Action|Horror,2.188035
1758271,89524,2734,4.0,1482247139,89523,2617,37,1972,"Mosquito Coast, The (1986)",Adventure|Drama|Thriller,3.665898
1758272,272360,3977,1.5,1511650186,272359,504,4568,228,Charlie's Angels (2000),Action|Comedy,1.781651


In [111]:
user_id = 118460

# trainで見ているmovieでratingの高いものTOP:20
train.query('userId == @user_id').sort_values('rating', ascending=False).head(20)

,userId,movieId,rating,timestamp,user,movie,user_idx,movie_idx,title,genres
3141036,118460,2571,5.0,1508005073,118459,376,4561,4,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2769265,118460,296,5.0,1508005203,118459,106,4561,2,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3502217,118460,48043,5.0,1508507418,118459,1318,4561,1413,"Fountain, The (2006)",Drama|Fantasy|Romance
1352415,118460,94959,5.0,1508005787,118459,1363,4561,915,Moonrise Kingdom (2012),Comedy|Drama|Romance
1139923,118460,2959,5.0,1508005062,118459,421,4561,12,Fight Club (1999),Action|Crime|Drama|Thriller
4031510,118460,356,5.0,1508005568,118459,121,4561,1,Forrest Gump (1994),Comedy|Drama|Romance|War
2036537,118460,112552,4.5,1508005051,118459,1045,4561,614,Whiplash (2014),Drama
2246023,118460,4011,4.5,1508005214,118459,513,4561,145,Snatch (2000),Comedy|Crime|Thriller
2314937,118460,1704,4.5,1508005069,118459,284,4561,45,Good Will Hunting (1997),Drama|Romance
1648679,118460,2858,4.5,1508005212,118459,410,4561,18,American Beauty (1999),Drama|Romance


In [112]:
test.query('userId == @user_id').sort_values('pred_rating', ascending=False).head(20)

,userId,movieId,rating,timestamp,user,movie,user_idx,movie_idx,title,genres,pred_rating
534653,118460,2329,3.5,1508005321,118459,770,4561,84,American History X (1998),Crime|Drama,4.005506
143396,118460,3275,4.0,1508005537,118459,456,4561,586,"Boondock Saints, The (2000)",Action|Crime|Drama|Thriller,3.777506
972553,118460,4973,5.0,1508005124,118459,774,4561,69,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance,3.758426
114281,118460,99114,4.5,1508006807,118459,1026,4561,230,Django Unchained (2012),Action|Drama|Western,3.735313
537700,118460,140174,3.5,1508007843,118459,1371,4561,1704,Room (2015),Drama,3.731752
1433574,118460,68157,4.5,1508005333,118459,972,4561,191,Inglourious Basterds (2009),Action|Drama|War,3.731267
720674,118460,288,4.5,1508507162,118459,103,4561,147,Natural Born Killers (1994),Action|Crime|Thriller,3.717692
649561,118460,3147,4.0,1508006357,118459,949,4561,90,"Green Mile, The (1999)",Crime|Drama,3.692951
1663474,118460,1884,2.5,1508507005,118459,300,4561,588,Fear and Loathing in Las Vegas (1998),Adventure|Comedy|Drama,3.678774
916856,118460,81591,3.0,1508005575,118459,985,4561,440,Black Swan (2010),Drama|Thriller,3.677341


In [124]:
_df = ratings_small[['movieId', 'movie_idx']].drop_duplicates()  # 重複を排除したデータ

def sim_movie(movie_df, movie_idx, asc=False, N=10):
    """類似映画IDTOP:N件を返却する関数
    """
    sim_movie_df = cos_df.iloc[:, movie_idx:movie_idx+1].sort_values(movie_idx, ascending=asc)[:N].reset_index().rename(columns={movie_idx: 'cos_sim', 'index': 'movie_idx'})
    sim_movie_df = pd.merge(sim_movie_df, _df, how='left', on='movie_idx')
    
    return pd.merge(sim_movie_df, movie_df, how='left', on='movieId')


def cos_sim_matrix(matrix):
    """コサイン類似度を計算する関数
    """
    d = matrix @ matrix.T  # item-vector 同士の内積を要素とする行列
    norm = (matrix * matrix).sum(axis=1, keepdims=True) ** .5
    return d / norm / norm.T

In [118]:
u_embedding = model.layers[2]
m_embedding = model.layers[3]

In [131]:
m_embedding.get_weights()[0]

array([[-0.9136246 , -0.6841883 ,  0.8554877 , ..., -0.35710463,
        -0.35618314,  0.00264137],
       [-0.7838982 , -0.60596853,  1.091007  , ..., -1.3012428 ,
        -0.86071956,  0.0875385 ],
       [-0.35396928, -1.1755496 , -0.18670538, ...,  0.29451543,
        -0.09152044, -0.6308714 ],
       ...,
       [-0.3434417 ,  0.4314484 ,  0.34840646, ..., -0.62360877,
        -0.25073776, -0.2673883 ],
       [ 0.50009584,  0.4552495 ,  0.05655844, ..., -0.08631565,
         0.2755157 ,  0.06652404],
       [-0.39525726,  0.05719721, -0.03015789, ...,  0.24584712,
        -0.4369471 , -1.0749834 ]], dtype=float32)

In [121]:
cos_df = cos_sim_matrix(m_embedding.get_weights()[0])
cos_df = pd.DataFrame(cos_df)
cos_df

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,1.000000,0.874368,0.427924,0.606093,0.578540,0.302607,0.419479,0.826611,0.872251,0.353316,...,0.338075,-0.001691,0.056492,-0.108682,0.403936,0.393109,-0.171766,0.280472,-0.407212,0.160833
1,0.874368,1.000000,0.233394,0.433959,0.430604,0.306978,0.491737,0.747624,0.868115,0.303843,...,0.164467,-0.116995,0.286231,-0.248419,0.329505,0.248643,-0.264203,0.365861,-0.207092,0.038618
2,0.427924,0.233394,1.000000,0.707119,0.486191,0.100060,0.119545,0.350855,0.320472,0.013287,...,0.303921,0.349060,0.012192,-0.458394,-0.139434,0.012659,-0.025169,-0.298057,-0.451502,0.397799
3,0.606093,0.433959,0.707119,1.000000,0.581497,0.277569,0.619782,0.668211,0.528455,0.456576,...,0.245288,0.039913,0.317465,-0.162019,-0.143103,0.430308,-0.277699,-0.257807,-0.455170,0.074451
4,0.578540,0.430604,0.486191,0.581497,1.000000,0.506234,0.573182,0.421508,0.675196,0.367704,...,0.153511,0.017121,-0.196335,-0.425414,0.063230,-0.135618,-0.285719,-0.018210,-0.379332,-0.122380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.393109,0.248643,0.012659,0.430308,-0.135618,0.009735,0.286820,0.421288,0.310087,0.184463,...,0.044137,0.260869,0.390872,0.206150,0.261014,1.000000,0.177477,-0.134428,-0.106587,-0.290008
1996,-0.171766,-0.264203,-0.025169,-0.277699,-0.285719,0.231231,-0.372888,-0.024654,-0.143553,-0.106506,...,-0.192654,0.806934,-0.461488,0.152602,-0.093570,0.177477,1.000000,-0.058380,-0.359823,-0.182761
1997,0.280472,0.365861,-0.298057,-0.257807,-0.018210,-0.220886,-0.315867,0.349809,0.266000,-0.156654,...,0.634741,-0.248322,-0.070334,0.316372,0.669660,-0.134428,-0.058380,1.000000,-0.389048,0.246460
1998,-0.407212,-0.207092,-0.451502,-0.455170,-0.379332,-0.274938,0.037102,-0.682696,-0.388526,-0.262501,...,-0.604698,-0.322134,0.316697,-0.213376,-0.102237,-0.106587,-0.359823,-0.389048,1.000000,-0.196503


In [129]:
sim_df = sim_movie(movies, movie_id_map[1], False, 20)
sim_df

,movie_idx,cos_sim,movieId,title,genres
0,9,1.000000,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,113,0.962630,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
2,32,0.917691,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
3,71,0.896649,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy
4,70,0.881967,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy
5,46,0.868783,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX
6,161,0.864197,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy
7,388,0.851554,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX
8,303,0.839457,2081,"Little Mermaid, The (1989)",Animation|Children|Comedy|Musical|Romance
9,101,0.829944,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy


In [137]:
u_bias = model.layers[5]
m_bias = model.layers[6]

In [145]:
m_bias.get_weights()[0]

array([[3.788693 ],
       [3.2956965],
       [3.6319456],
       ...,
       [3.004461 ],
       [1.9291176],
       [2.768853 ]], dtype=float32)

In [150]:
model.layers[7]